In [14]:
import matplotlib        as mpl
import matplotlib.pyplot as plt
import numpy             as np
import pandas            as pd
import yaml
import netCDF4
import sys

sys.path.append('/home/veilleux/Projet/Projet_SACHR/analyses_and_figures/python_modules')
from domain         import generate_domain_coord
from domain         import convert_latlon_to_domain_indices
from satellite      import extract_satellite_track

NaN = np.nan
import warnings
warnings.filterwarnings("ignore")

In [15]:
def convert_calipso_data_in_2D(ncfile, varname, coord_domain, ni, nj):

    track   = extract_satellite_track(ncfile, coord_domain, 'calipso')
    indices = convert_latlon_to_domain_indices(track, 'NAM11')

    nc      = netCDF4.Dataset(ncfile,'r')
    data    = nc[varname][track['index']]

    data_sum  = np.ones((655,655)) * 0
    data_n    = np.ones((655,655)) * 0
    for i in range(len(indices['i'])): 
        I = indices['i'][i]
        J = indices['j'][i]
        data_sum[I, J] = data_sum[I, J] + data[i]
        data_n[  I, J] = data_n[  I, J] + 1
    return data_sum/data_n

In [17]:
 #########################################################################
# Input arguments                                                       #
#########################################################################

working_directory = '/home/veilleux/Projet/Projet_SACHR/analyses_and_figures'
YYYYi             = int(2014)
YYYYf             = int(2015)
period            = 'DJF'
window            = int(25)
layerdef          = 'pmaxran'
#########################################################################
# Hardcoded values                                                      #
#########################################################################
if   period == 'annual': MMs = [1,2,3,4,5,6,7,8,9,10,11,12]
elif period == 'DJF'   : MMs = [ 1, 2,12]
elif period == 'MAM'   : MMs = [ 3, 4, 5]
elif period == 'JJA'   : MMs = [ 6, 7, 8]
elif period == 'SON'   : MMs = [ 9,10,11]
else                   : MMs = [ int(period) ] # single month

#########################################################################
# Configuration file (yml)                                              #
#########################################################################
yml_file = working_directory + '/config.yml'
stream = open(yml_file,'r')
config = yaml.safe_load(stream)

dirlist = config['CALIPSO_CHINOOK']['list']

### COMPLETE filelist 2014-2015
filelist   = dirlist + '/' + '2014_2015.txt'
df         = pd.read_csv(filelist,delimiter='\s+', header=None)
df.columns = ['file', 'ndata', 'ti', 'tf', 'date', 'MM', 'date_gem', 't_gem']
df         = df[   np.isin(df['MM'], MMs)  ].reset_index(drop=True)


In [26]:
gemname              = "COSP2_NAM-11m_ERA5_GEM5_CLASS_NEWVEG_newP3-SCPF_SN_Lakes"
pm                   = "pm2013010100"
# CRCM6-GEM5 filepath
GEM5_filepath    = config['GEM5']['NetCDF']+'/{}_YYYYMM/{}_YYYYMMDDd.nc'.format(gemname,pm)
#COSP2(CRCM6) filepath
COSP2_filepath   = config['COSP2']['output']+ '/NAM11/M01SC002/CALIPSO/calipso_cloudprofile/YYYYMM/cospout_YYYYMMDDHH00_2D.nc'